
The current script makes changes on the raw load demand data to create synthetic scenario-based inputs.

It incorporates 4 demand scenarios (O = 1:4), each designed to reflect different system stress conditions:

- O = 1: Baseline demand (original profile)
- O = 2: Stress scenario (+15% demand, capped at 1.0)
- O = 3: Low demand scenario (−15%)
- O = 4: Noisy scenario (±5% random variation)

This script preserves the time structure of the original data (T = 1:720).


In [1]:
using DataFrames, CSV, Random
#Change the name of the excel file
df = CSV.read("load_profile.csv", DataFrame)

Row,T,O,value
,Int64,Int64,Float64
1,1,1,0.75
2,2,1,0.6
3,3,1,0.86
4,4,1,0.88
5,5,1,0.55
6,6,1,0.82
7,7,1,0.75
8,8,1,0.91
9,9,1,0.64


In [2]:
# Extract the base profile (assumes scenario 1 is the baseline)
base = df[df.O .== 1, :]
base = sort(base, :T)

# Generate synthetic scenarios
T_vals = base.T
N = length(T_vals)
base_vals = base.value

# Create new scenarios
stress_vals = min.((1.15 .* base_vals), 1.0)                      # O = 2
low_vals = 0.85 .* base_vals                                      # O = 3
noisy_vals = clamp.((base_vals .* (1 .+ rand(N) .* 0.1 .- 0.05)), 0.0, 1.0)  # O = 4
# Assemble new DataFrame
new_df = DataFrame(T = repeat(T_vals, 4),
                   O = repeat(1:4, inner = N),
                   value = vcat(base_vals, stress_vals, low_vals, noisy_vals))

# Save to file
CSV.write("load_profile.csv", new_df)

"load_profile.csv"